In [4]:
import sys
import os 
sys.path.insert(1, os.path.split(os.getcwd())[0])

In [5]:
from datasets.dataloader import DataLoader
from model import Trainer
from utils.utils import create_folders
from batch_gen import BatchGenerator

import torch 

from asformer import MyTransformer, ASFormerTrainer

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [7]:
class Args(): 
    def __init__(self, *args, **kwargs):
        self.train_data = 'bslcp'
        self.test_data = 'bslcp'
        self.i3d_training = 'i3d_kinetics_bslcp_981'
        self.num_in_frames = 16
        self.features_dim = 1024
        self.weights = 'opt'
        self.regression = 0 
        self.feature_normalization = 0
        self.eval_use_CP = 0

        self.action = 'train'
        self.seed = 0 
        self.refresh = 'store_true'

        ## Transformer : 
        self.nhead = 4
        self.nhid = 1024
        self.dim_feedforward = 1024
        self.num_layers = 6
        self.dropout = 0

        ## MSTCN : 
        self.num_stages = 4
        self.num_layers = 10 
        self.num_f_maps = 64
        self.features_dim = 1024
        self.bz = 8 
        self.lr = 0.0005
        self.lr_mul = 1
        self.num_epochs = 50
        self.extract_epoch = 10 
        self.classification_threshold = 0.5

        ## ASFormer : 
        self.num_layers = 7
        self.num_decoders = 3
        self.r1 = 2
        self.r2 = 2
        self.channel_masking_rate = 0.3
        self.input_dim = 1024
        self.num_f_maps = 64
        
        
        ## Optimization
        self.n_warmup_steps = 100
        ## save model : 
        self.use_pseudo_labels = 'store_true'
        self.pretrained = False
        self.folder = ''
        
args = Args()

# Dataloader

In [8]:
# load train dataset and test dataset

print(f'Load train data: {args.train_data}')
train_loader = DataLoader(args, args.train_data, 'train')
print(f'Load test data: {args.test_data}')
test_loader = DataLoader(args, args.test_data, 'test')

Load train data: bslcp
Load test data: bslcp


In [9]:
model_load_dir, model_save_dir, results_save_dir = create_folders(args)
print(model_save_dir)

Saved options to ./exps//models/classification/traindata_bslcp/i3d_kinetics_bslcp_981/supervised/7_(7,)_(64,)_8_0.0005_weighted_opt/seed_0\opt.txt
./exps//models/classification/traindata_bslcp/i3d_kinetics_bslcp_981/supervised/7_(7,)_(64,)_8_0.0005_weighted_opt/seed_0


# AS Former model : 

In [11]:
trainer = ASFormerTrainer(args.num_decoders,args.num_layers, args.r1, args.r2, args.num_f_maps, args.input_dim, 2, args.channel_masking_rate, train_loader.weights, model_save_dir)

Model Size:  762888


Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\threading.py", line 926, in _bootstrap_inner
    self.run()
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\site-packages\tensorboard\summary\writer\event_file_writer.py", line 238, in run
    self._record_writer.write(data)
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\site-packages\tensorboard\summary\writer\record_writer.py", line 40, in write
    self._writer.write(header + header_crc + data + footer_crc)
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\site-packages\tensorboard\compat\tensorflow_stub\io\gfile.py", line 531, in write
    self.fs.write(self.filename, file_content, self.binary_mode)
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\site-packages\tensorboard\compat\tensorflow_stub\io\gfile.py", line 148, in write
    self._write(filename, file_content, "wb" if binary_mode else "w")
  File "C:\Users\kam_a\.conda\envs\signseg_env\lib\site-packag

In [12]:
eval_args = [
    args,
    model_save_dir,
    results_save_dir,
    test_loader.features_dict,
    test_loader.gt_dict,
    test_loader.eval_gt_dict,
    test_loader.vid_list,
    args.num_epochs,
    device,
    'eval',
    args.classification_threshold,
]

In [13]:
batch_gen = BatchGenerator(
        train_loader.num_classes,
        train_loader.gt_dict,
        train_loader.features_dict,
        train_loader.eval_gt_dict
        )

batch_gen.read_data(train_loader.vid_list)

In [14]:
trainer.train(model_save_dir, batch_gen, 15, 1, 0.0005, eval_args)

LR:0.0005


KeyboardInterrupt: 